In [1]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
app_name = "add_thesis"
%run $header_path $app_name=app_name

C:\Users\common\projet_3_lru\sorting_univ\script/headers/
add_thesis


In [2]:
df_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + '\\publications.parquet')
set_french_authors = spark.read.parquet('file:\\' + save_path + 'set_french_authors.parquet')

merge_path = databases_path + '\\merge\\'

merge_authors = spark.read.format('parquet').load('file:\\' + merge_path + 'matched_authors_2steps.parquet').withColumnRenamed('id','idref')

In [4]:
set_french_authors.join(merge_authors.groupBy("author_id").agg(func.concat_ws(',', func.collect_set('idref')).alias('idref')), on ='author_id', how = 'inner').count()

90189

In [5]:
merge_authors.filter(func.col('author_id')=="A5084984675").show()

+-----+---------+
|idref|author_id|
+-----+---------+
+-----+---------+



In [4]:
df_au_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + '/authors.parquet')


In [5]:
df_au_scanR.filter(func.col('id')=="idref078039886").show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+---------+-----------------+------+--------------+------+---------+---------+-------------------+--------------------+-----------------+--------------------+--------------------+
|        affiliations|       autocompleted|   autocompletedText|awards|      coContributors|             domains|         externalIds|firstName|         fullName|gender|            id|id_hal|    idref| lastName|              orcid|        publications|publicationsCount|  recentAffiliations|          topDomains|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+---------+-----------------+------+--------------+------+---------+---------+-------------------+--------------------+-----------------+--------------------+--------------------+
|[{2018-12-31T00:0...|[Le Bellec, 07803...|[Le Bellec, 07803.

In [7]:
merged_authors = spark.read.parquet('file:\\' + merge_path + 'merge_oa_scanr_authors.parquet')

+---+---------+
| id|author_id|
+---+---------+
+---+---------+



In [8]:
spark.stop()

In [3]:
merge_authors = merge_authors.withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', ''))

In [13]:
theses_oa = (df_scanR
          .filter(func.col('type')=='thesis')
          .withColumn('authors', func.explode(func.col('authors')))
          .withColumn('idref', func.col('authors.person'))
          .withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', ''))
         .select('idref','authors.role','id','authors.affiliations', 'authors.firstName', 'authors.lastName', 'authors.fullName')
         .join(merge_authors, on = 'idref', how= 'inner')
         )
theses_oa.cache()

DataFrame[idref: string, role: string, id: string, affiliations: array<struct<BNF:string,ISNI:string,acronym:array<string>,address:string,address-line:string,addresses:array<string>,alias_idref:array<string>,aliases:array<string>,city:string,comments:array<string>,country:string,datasource:string,department:array<string>,detected_countries:array<string>,docid:string,external_ids:array<struct<id_type:string,id_value:string>>,grid:array<string>,hal_docid:string,id:array<struct<asserted-by:string,id:string,id-type:string>>,idref:string,ids:array<struct<id:string,type:string>>,name:string,orgs:array<struct<name:string>>,place:array<string>,postal-code:string,rnsr:string,ror:string,structId:string,viaf:string,websites:array<string>,zipcode:string>>, firstName: string, lastName: string, fullName: string, author_id: string]

In [ ]:
theses_oa.show()

In [5]:
#(theses_oa.groupBy('id')
# .agg(func.max( (func.col('role')=="directeurthese").cast('int')).alias('has_supervisor'),
#      func.max( (func.col('role')=="rapporteur").cast('int')).alias('has_ref'),
#      func.max( (func.col('role')=="membrejury").cast('int')).alias('has_jury'))
#.groupBy('has_supervisor','has_ref','has_jury').count()).show()

+--------------+-------+--------+------+
|has_supervisor|has_ref|has_jury| count|
+--------------+-------+--------+------+
|             0|      1|       1| 14098|
|             0|      0|       0| 47368|
|             1|      0|       1| 24020|
|             1|      1|       1| 91284|
|             1|      0|       0|192103|
|             1|      1|       0| 10280|
|             0|      1|       0|  2756|
|             0|      0|       1|  6245|
+--------------+-------+--------+------+



In [8]:
merged = (set_french_authors.join(theses_oa, how= 'left', on =['author_id']))
merged.cache()
merged.groupBy(func.col('id').isNotNull().cast('int').alias('has_link_these')).count().show()

+--------------+-------+
|has_link_these|  count|
+--------------+-------+
|             1|1030483|
|             0| 220060|
+--------------+-------+



In [11]:
merged.filter(func.col('idref').isNotNull()).select('author_id').distinct().count()

160286

In [6]:
(set_french_authors.join(theses_oa.filter(func.col('role')=='author'), how= 'left', on =['author_id']).groupBy(func.col('id').isNotNull().cast('int').alias('has_these')).count()).show()

+---------+------+
|has_these| count|
+---------+------+
|        1|138202|
|        0|249054|
+---------+------+



In [8]:
(set_french_authors.join(theses_oa.filter(func.col('role')=='directeurthese'), how= 'left', on =['author_id']).groupBy(
    'author_id'
).agg(func.max(func.col('id').isNotNull().cast('int')).alias('has_supervised')).groupBy('has_supervised').count()).show()

+--------------+------+
|has_supervised| count|
+--------------+------+
|             1| 58018|
|             0|322328|
+--------------+------+

